In [0]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras import callbacks
from keras.optimizers import Adam
import time

In [0]:
start=time.time()

In [0]:
train_data_path = 'data/train'
validation_data_path = 'data/test'
img_width, img_height = 224, 224
filters_layer1=64
conv_size=3
pool_size1=2
stride_size=2

filters_layer2=128

filters_layer3=256

filters_layer4=512
dense_units=4096
classes_num=15
batch_size=89600

model = Sequential()
model.add(Conv2D(input_shape=(img_width, img_height,3),filters=filters_layer1,kernel_size=(conv_size,conv_size),
                 padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer1,kernel_size=(conv_size,conv_size),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(pool_size1,pool_size1),strides=(stride_size,stride_size)))

model.add(Conv2D(filters=filters_layer2, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer2, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(pool_size1,pool_size1),strides=(stride_size,stride_size)))

model.add(Conv2D(filters=filters_layer3, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer3, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer3, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(pool_size1,pool_size1),strides=(stride_size,stride_size)))

model.add(Conv2D(filters=filters_layer4, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer4, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer4, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(pool_size1,pool_size1),strides=(stride_size,stride_size)))

model.add(Conv2D(filters=filters_layer4, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer4, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(Conv2D(filters=filters_layer4, kernel_size=(conv_size,conv_size), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(pool_size1,pool_size1),strides=(stride_size,stride_size)))

model.add(Flatten())
model.add(Dense(units=dense_units,activation="relu"))
model.add(Dense(units=dense_units,activation="relu"))
model.add(Dense(units=classes_num, activation="softmax"))

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 56, 56, 256)       295168    
__________

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

traindata = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

testdata = ImageDataGenerator(rescale=1. / 255)

train_generator = traindata.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = testdata.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

checkpoint = ModelCheckpoint("CNN.best.from_scratch.hdf5", monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

hist = model.fit_generator(steps_per_epoch=batch_size,
                           generator=train_generator,
                           validation_data= validation_generator,
                           validation_steps=10,
                           epochs=20,
                           callbacks=[checkpoint,early])
target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir) 

model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Train on 89600 samples, validate on 11200 samples
Epoch 1/20
89600/89600 [==============================] - 4285s 48ms/step - loss: 0.6572 - acc: 0.6144 - val_loss: 0.6502 - val_acc: 0.6272

Epoch 00001: val_loss improved from inf to 0.65022, saving model to CNN.best.from_scratch.hdf5
Epoch 2/20
89600/89600 [==============================] - 4196s 47ms/step - loss: 0.6454 - acc: 0.6349 - val_loss: 0.6470 - val_acc: 0.6339

Epoch 00002: val_loss improved from 0.65022 to 0.64695, saving model to CNN.best.from_scratch.hdf5
Epoch 3/20
89600/89600 [==============================] - 4163s 46ms/step - loss: 0.6419 - acc: 0.6394 - val_loss: 0.6399 - val_acc: 0.6403

Epoch 00003: val_loss improved from 0.64695 to 0.63990, saving model to CNN.best.from_scratch.hdf5
Epoch 4/20
89600/89600 [==============================] - 4207s 47ms/step - loss: 0.6384 - acc: 0.6458 - val_loss: 0.6358 - val_acc: 0.6479

Epoch 00004: val_loss improved from 0.63990 to 0.63585, saving model to CNN.best.from_scratch